In [37]:
# require at least Python 3.5 for async/await to work
import sys
ver=sys.version_info
assert (ver[0]>=3 and ver[1]>=5)
sys.path.append('../../quant_container/src')
sys.path.append('../src')
%load_ext autoreload
%autoreload 2
from mosaicsmartdata.core.fx import FXPricingContextGenerator
from mosaicsmartdata.common import qc_csv_helper
from mosaicsmartdata.core.quote import Quote
import csv, copy
import pickle, cloudpickle

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
try:
    with open('data.pickle', 'rb') as f:
        # Pickle the 'data' dictionary using the highest protocol available.
        quote_dict = pickle.load(f)
except:
#if True:
    file_path ='../resources/fx/'
    filenames = ['OIS-N156898469.csv','EUR_JPY_GBP_till_SN.csv','EUR_JPY_GBP_from_SW_till_1Y.csv']
    quote_dict_0 = qc_csv_helper.get_ric_quotes(file_path+filenames[0])
    print(len(quote_dict_0), quote_dict_0.keys())
    quote_dict_1 = qc_csv_helper.get_ric_quotes(file_path+filenames[1])
    print(len(quote_dict_1), quote_dict_1.keys())
    quote_dict_2 = qc_csv_helper.get_ric_quotes(file_path+filenames[2])
    print(len(quote_dict_2), quote_dict_2.keys())
    quote_dict = dict(quote_dict_0,**quote_dict_1)
    quote_dict = dict(quote_dict,**quote_dict_2)
    with open('data.pickle', 'wb') as f:
        pickle.dump(quote_dict, f)

In [3]:
quote_dict['EURSN='][0].instrument.settle_date

datetime.date(2017, 6, 28)

In [4]:
# create a trade
from mosaicsmartdata.core.instrument_utils import sym_to_instrument
from mosaicsmartdata.core.trade import Trade, FXForwardTrade

#spot_quotes = [quotes for sym, quotes in quote_dict.items() if len(sym)==4]
my_quote = quote_dict['EUR='][100]
my_instr = sym_to_instrument()(my_quote.sym, my_quote.timestamp.date())
my_instr.notionals = (100, -100*my_quote.bid)
print(my_instr)
print('*****')
my_trade = FXForwardTrade(my_instr, 
                          trade_id = '12345',
                        timestamp = my_quote.timestamp)
print(my_trade)
print('*******')
print(my_quote)

<class 'mosaicsmartdata.core.instrument.FXForward'> :  notionals = (100, -111.97999999999999), pip_size = 0.0001, tenor = SPOT, static = Instrument static singleton facade, sym = EUR=, venue = None, maturity_date = 2017-06-28, isDeliverable = True, holidayCities = None, ccy = ('EUR', 'USD'), date_calc = <mosaicsmartdata.core.date_calculator.DateCalculator object at 0x0000021BAA72E048>, settle_date = 2017-06-28
*****
<class 'mosaicsmartdata.core.trade.FXForwardTrade'> :  trade_rc = None, factor_risk = None, timestamp = 2017-06-26 00:02:26.995000, orig_package_size = None, trader = None, source_trade_id = None, package_id = 12345, package = None, trade_id = 12345, instrument = <class 'mosaicsmartdata.core.instrument.FXForward'> :  notionals = (100, -111.97999999999999), pip_size = 0.0001, tenor = SPOT, static = Instrument static singleton facade, sym = EUR=, venue = None, maturity_date = 2017-06-28, isDeliverable = True, holidayCities = None, ccy = ('EUR', 'USD'), date_calc = <mosaicsmar

In [5]:
from aiostreams import run
import aiostreams.operators as op
from mosaicsmartdata.core.markout import MarkoutCalculator
import logging

In [6]:
# run markouts based on simple instrument ID, with no pricing context
# logging.getLogger().setLevel('INFO')
# logging.info('starting command line test...')

# run the markout calc
trades = [my_trade]
quotes = quote_dict['EUR=']
stream = op.merge_sorted([quotes[:1000], trades], lambda x: x.timestamp) \
            | op.flat_map_by_group(lambda x: x.sym, MarkoutCalculator([-10, 0, 10, 100]))\
            > print
        
run(stream)

 dt:-10 final_price:1.1199 hedged_bps:None hedged_cents:None hedged_price:None initial_price:1.1198 next_timestamp:2017-06-26 00:02:16.995000 package:None price:9.999999999998899e-05 PV:9.999999999998899e-05 timestamp:2017-06-26 00:02:26.995000 package_id:12345 trade_id:12345 sym:EUR= timestamp:26/06/2017 00:02:26 delta:1.0 paper_trade:False client_sys_key:None notional:None trade_maturity_date:28/06/2017 traded_px:1.1198 ccy:('EUR', 'USD')
 dt:0 final_price:1.11985 hedged_bps:None hedged_cents:None hedged_price:None initial_price:1.1198 next_timestamp:2017-06-26 00:02:26.995000 package:None price:5.0000000000105516e-05 PV:5.0000000000105516e-05 timestamp:2017-06-26 00:02:27.165000 package_id:12345 trade_id:12345 sym:EUR= timestamp:26/06/2017 00:02:26 delta:1.0 paper_trade:False client_sys_key:None notional:None trade_maturity_date:28/06/2017 traded_px:1.1198 ccy:('EUR', 'USD')
 dt:10 final_price:1.11985 hedged_bps:None hedged_cents:None hedged_price:None initial_price:1.1198 next_time

In [7]:
# run markouts using PricingContext
# run the markout calc

quotes_eur = quote_dict['EUR=']
quotes_jpy = quote_dict['JPY=']
print(len(quotes_jpy),len(quotes_eur))
quotes_all = op.merge_sorted([quotes_eur[:1000],quotes_jpy[:1000]], lambda x: x.timestamp) > []
             #|\op.map(lambda x: (x.sym, x.timestamp)) > 
run(quotes_all)

my_quote = quotes_all.sink[100]
my_instr_2 = sym_to_instrument()('EURJPY=', my_quote.timestamp.date())
my_instr_2.notionals = (100, -100*124.594511) # just picked a random quote
print(my_instr_2) 
print('*****')
my_trade_2 = FXForwardTrade(my_instr_2, 
                          trade_id = '12346',
                        timestamp = my_quote.timestamp)
print(my_trade_2.timestamp)


553108 591602
<class 'mosaicsmartdata.core.instrument.FXForward'> :  notionals = (100, -12459.4511), pip_size = 0.0001, tenor = SPOT, static = Instrument static singleton facade, sym = EURJPY=, venue = None, maturity_date = 2017-06-28, isDeliverable = True, holidayCities = None, ccy = ('EUR', 'JPY'), date_calc = <mosaicsmartdata.core.date_calculator.DateCalculator object at 0x0000021BAD14B908>, settle_date = 2017-06-28
*****
2017-06-26 00:00:53.188000


In [8]:
#markouts using PricingContext
trades = [my_trade]
quotes_2 = quotes_all.sink
#print([quote.sym for quote in quotes_2][:100])

prices =  quotes_2 | op.map(FXPricingContextGenerator()) 
#stream = prices | op.map(lambda x: x.timestamp) > print
stream = op.merge_sorted([trades,prices], lambda x:x.timestamp)\
        | op.flat_map(MarkoutCalculator([-10,0, 10, 100])) | op.map(lambda x:str(x))\
        > []
run(stream)       

stream.sink

'list' object has no attribute 'timestamp'
Traceback (most recent call last):
  File "../../quant_container/src\aiostreams\runner.py", line 161, in check_tasks
    t.result()
  File "C:\Users\egork\Anaconda3\envs\py35\lib\asyncio\futures.py", line 293, in result
    raise self._exception
  File "C:\Users\egork\Anaconda3\envs\py35\lib\asyncio\tasks.py", line 239, in _step
    result = coro.send(None)
  File "../../quant_container/src\aiostreams\runner.py", line 58, in coro
    async for msg in self.source:
  File "../../quant_container/src\aiostreams\operators\map.py", line 12, in __anext__
    val = await self.aiter.__anext__()
  File "../../quant_container/src\aiostreams\operators\flatten.py", line 18, in __anext__
    self.latest = await self.aiter.__anext__()
  File "../../quant_container/src\aiostreams\operators\map.py", line 12, in __anext__
    val = await self.aiter.__anext__()
  File "../../quant_container/src\aiostreams\operators\merge_sorted.py", line 56, in __anext__
    tes

[]

In [41]:
# extract just the first quote from every stream
tiny_quote_dict = {key:[quotes[0]] for key, quotes in quote_dict.items()}
print([(key,quotes[0].bid, quotes[0].instrument.ccy) for key, quotes in tiny_quote_dict.items()])
with open('../resources/fx/data.pickle', 'wb') as f:
        pickle.dump(tiny_quote_dict, f)


[('GBP1Y=', 141.19, ('GBP', 'USD')), ('GBP3M=', 36.12, ('GBP', 'USD')), ('EUR6M=', 110.79, ('EUR', 'USD')), ('USD1MOIS=', 1.141, 'USD'), ('JPY=', 111.22, ('USD', 'JPY')), ('GBP6M=', 70.3, ('GBP', 'USD')), ('EURSN=', 0.54, ('EUR', 'USD')), ('JPY1M=', -16.39, ('USD', 'JPY')), ('USD6MOIS=', 1.175, 'USD'), ('JPYSW=', -5.7, ('USD', 'JPY')), ('JPY6M=', -93.41, ('USD', 'JPY')), ('GBPTN=', 0.385, ('GBP', 'USD')), ('EUR1M=', 19.11, ('EUR', 'USD')), ('JPY3M=', -45.71, ('USD', 'JPY')), ('JPYTN=', -0.481, ('USD', 'JPY')), ('GBP1M=', 13.74, ('GBP', 'USD')), ('EURON=', 0.525, ('EUR', 'USD')), ('USD3MOIS=', 1.148, 'USD'), ('EUR1Y=', 230.8, ('EUR', 'USD')), ('GBPON=', 0.352, ('GBP', 'USD')), ('GBPSW=', 4.53, ('GBP', 'USD')), ('EUR3M=', 54.91, ('EUR', 'USD')), ('EUR=', 1.1198, ('EUR', 'USD')), ('JPYON=', -0.47, ('USD', 'JPY')), ('GBPSN=', 0.387, ('GBP', 'USD')), ('JPYSN=', -0.466, ('USD', 'JPY')), ('USDSWOIS=', 1.11, 'USD'), ('EURSW=', 5.84, ('EUR', 'USD')), ('JPY1Y=', -202.86, ('USD', 'JPY')), ('GBP='

In [10]:
import datetime
a = datetime.date(2107,1,1)
b = datetime.date(2107,1,2)
from mosaicsmartdata.common.quantlib.bond.fixed_bond import pydate_to_qldate
pydate_to_qldate(b)< pydate_to_qldate(a)


False

In [44]:
with open('../resources/fx/data.pickle', 'rb') as f:
        tiny_quote_dict = pickle.load(f)

        
tiny_quotes_list = [quotes for key, quotes in tiny_quote_dict.items()]
print([q[0].instrument.sym for q in tiny_quotes_list])
stream = op.merge_sorted(tiny_quotes_list, lambda x: x.timestamp) |\
            op.flat_map(FXPricingContextGenerator()) > []
run(stream)
pc = stream.sink[-1]
pc.extra


['GBP1Y=', 'GBP3M=', 'EUR6M=', 'USD1MOIS=', 'JPY=', 'USD3MOIS=', 'EURSN=', 'JPY1M=', 'USD6MOIS=', 'JPYSW=', 'JPY6M=', 'GBPTN=', 'EUR1M=', 'JPY3M=', 'GBPSN=', 'GBP1M=', 'EURON=', 'GBP6M=', 'EUR1Y=', 'GBPON=', 'GBPSW=', 'EUR3M=', 'EUR=', 'JPYON=', 'JPYTN=', 'JPYSN=', 'USDSWOIS=', 'EURSW=', 'JPY1Y=', 'GBP=', 'EURTN=', 'USD1YOIS=']


{'EUR': {datetime.date(2017, 6, 26): 0.892983515970787,
  datetime.date(2017, 6, 27): 0.8929404573908906,
  datetime.date(2017, 6, 29): 0.8928534757803674,
  datetime.date(2017, 7, 5): 0.8924236799268926,
  datetime.date(2017, 7, 28): 0.891368080643853,
  datetime.date(2017, 9, 28): 0.888512159733162,
  datetime.date(2017, 12, 28): 0.8840939562011013,
  datetime.date(2018, 6, 28): 0.8747911436144621},
 'GBP': {datetime.date(2017, 6, 26): 0.7847012756025467,
  datetime.date(2017, 6, 27): 0.7846771387062009,
  datetime.date(2017, 6, 29): 0.7846271455727375,
  datetime.date(2017, 7, 5): 0.7843678622524536,
  datetime.date(2017, 7, 28): 0.7838010274063868,
  datetime.date(2017, 9, 28): 0.7824169014569385,
  datetime.date(2017, 12, 28): 0.780313014762742,
  datetime.date(2018, 6, 28): 0.7759384587689582},
 'JPY': {datetime.date(2017, 6, 26): 111.24919500000001,
  datetime.date(2017, 6, 27): 111.2445,
  datetime.date(2017, 6, 29): 111.235505,
  datetime.date(2017, 7, 5): 111.18520000000001,


In [43]:
from mosaicsmartdata.core.curve_utils import construct_OIS_curve, discounting_factor
import datetime

with open('EUR_curve.pickle', 'rb') as f:
        EUR_df = pickle.load(f)
EUR_df[(datetime.date(2017, 6, 27), datetime.date(2017, 6, 28))] = 1
[print(key, value,'\n') for key,value in EUR_df.items()]
        
eur_curve =  construct_OIS_curve(EUR_df)
df = discounting_factor(eur_curve,a,b)

(datetime.date(2017, 6, 28), datetime.date(2017, 12, 28)) 1.15646526169 

(datetime.date(2017, 6, 27), datetime.date(2017, 6, 28)) 1 

(datetime.date(2017, 6, 26), datetime.date(2017, 6, 28)) -0.0174391232534 

(datetime.date(2017, 6, 28), datetime.date(2018, 6, 28)) 1.22111758501 

(datetime.date(2017, 6, 28), datetime.date(2017, 7, 5)) 1.10253256208 

(datetime.date(2017, 6, 28), datetime.date(2017, 7, 28)) 1.12162081528 

(datetime.date(2017, 6, 28), datetime.date(2017, 6, 29)) 1.13257537723 

(datetime.date(2017, 6, 28), datetime.date(2017, 9, 28)) 1.13030866308 



RuntimeError: negative time (-0.00273973) given

In [14]:
for sym, quote in tiny_quote_dict.items():
    if 'OIS' not in sym and 'ON' not in sym: #and 'TN' not in sym:
            print(sym)
            print(quote[0].mid, quote[0].instrument.price(pc))


GBP1Y=
143.12 143.12
GBP3M=
36.41 36.41
EUR6M=
111.515 111.515000016
JPY=
111.24000000000001 111.24000000000001
EURSN=
0.546 0.546000000037
JPY1M=
-16.13 -16.1299999999
JPYSW=
-5.48 -5.47999999992
JPY6M=
-92.8 -92.8000000144
GBPTN=


RuntimeError: negative time (-0.00273973) given

In [15]:
pc.extra['EUR']
for sym, quote in tiny_quote_dict.items():
    if 'EUR' in sym:
        print(quote[0].mid, quote[0].instrument.tenor, quote[0].instrument.sym)
print(pc.extra['GBP'])

111.515 6M EUR6M=
0.546 1b EURSN=
19.21 1M EUR1M=
0.54 ON EURON=
231.8 1Y EUR1Y=
55.269999999999996 3M EUR3M=
1.11995 SPOT EUR=
5.9399999999999995 1w EURSW=
0.5449999999999999 TN EURTN=
{datetime.date(2017, 9, 28): 0.7824169014569385, datetime.date(2018, 6, 28): 0.7759384587689582, datetime.date(2017, 7, 28): 0.7838010274063868, datetime.date(2017, 7, 5): 0.7843678622524536, datetime.date(2017, 6, 29): 0.7846271455727375, datetime.date(2017, 12, 28): 0.780313014762742, datetime.date(2017, 6, 27): 0.7846771387062009}
